In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, label, find_objects
from PIL import Image
import cv2
import os

# This script analyzes Sentinel-1 SAR images from May and June 2022 to detect changes,
# creates annotated overlays on a context image, and generates a markdown intelligence report.

# === STEP 0: File Paths ===
may_sar_path = "../data/satellite_images/solonytsivka_may2022_sar.png"
jun_sar_path = "../data/satellite_images/solonytsivka_jun2022_sar.png"
context_image_path = "../data/satellite_images/solonytsivka_jun2022_falsecolorurban.png"  # or truecolor
output_overlay_path = "../data/output/change_maps/sar_overlay_blended.png"
report_output_path = "../reports/sar_intelligence_report.md"

# === STEP 1: Load SAR images (grayscale) ===
may_sar = np.array(Image.open(may_sar_path).convert("L")).astype(np.float32)
jun_sar = np.array(Image.open(jun_sar_path).convert("L")).astype(np.float32)

# === STEP 2: Normalize and smooth images ===
def normalize(img):
    img = np.clip(img, np.percentile(img, 5), np.percentile(img, 95))
    return (img - img.min()) / (img.max() - img.min())

may_sar_norm = normalize(gaussian_filter(may_sar, sigma=1))
jun_sar_norm = normalize(gaussian_filter(jun_sar, sigma=1))

# === STEP 3: Compute SAR difference ===
sar_diff = np.abs(jun_sar_norm - may_sar_norm)

# === STEP 4: Create color heatmap ===
sar_diff_8bit = (sar_diff * 255).astype(np.uint8)
heatmap = cv2.applyColorMap(sar_diff_8bit, cv2.COLORMAP_INFERNO)

# === STEP 5: Load context image and blend ===
context_img = cv2.imread(context_image_path)
context_img = cv2.resize(context_img, (heatmap.shape[1], heatmap.shape[0]))
blended = cv2.addWeighted(context_img, 0.7, heatmap, 0.6, 0)

# === STEP 6: Threshold for change detection ===
threshold = 0.2
mask = (sar_diff > threshold).astype(np.uint8)
labeled, n = label(mask)
regions = find_objects(labeled)

# === STEP 7: Annotate changes ===
annotations = []
for i, reg in enumerate(regions):
    y1, y2 = reg[0].start, reg[0].stop
    x1, x2 = reg[1].start, reg[1].stop
    area = (x2 - x1) * (y2 - y1)
    if area < 150:
        continue
    cv2.rectangle(blended, (x1, y1), (x2, y2), (0, 0, 255), 2)
    cv2.putText(blended, f"SAR Change {i+1}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
    annotations.append({
        "label": f"SAR Change {i+1}",
        "x1": x1, "y1": y1, "x2": x2, "y2": y2, "area": area
    })

# === STEP 8: Save overlay image ===
os.makedirs(os.path.dirname(output_overlay_path), exist_ok=True)
cv2.imwrite(output_overlay_path, blended)
print(f"[+] Blended SAR overlay saved to: {output_overlay_path}")

# === STEP 9: Generate Markdown Intelligence Report ===
os.makedirs(os.path.dirname(report_output_path), exist_ok=True)
with open(report_output_path, "w") as f:
    f.write("# SAR-Based Intelligence Report – Solonytsivka, Ukraine\n\n")
    f.write("## Summary\n")
    f.write("Radar backscatter comparison (Sentinel‑1 VV) between May and June 2022 reveals surface changes possibly indicating:\n\n")
    f.write("- Track marks from vehicle movement\n")
    f.write("- Construction or demolition\n")
    f.write("- Burned or disturbed ground\n\n")

    f.write("## Detected Changes\n")
    for a in sorted(annotations, key=lambda x: -x['area'])[:5]:
        f.write(f"- **{a['label']}** — Coords: ({a['x1']},{a['y1']})–({a['x2']},{a['y2']}), Area: {a['area']} px\n")

    f.write("\n---\n")
    f.write(f"Blended overlay saved as `{os.path.basename(output_overlay_path)}`.\n")
    f.write("Generated via `sar_analysis_change_detect.ipynb`.\n")

print(f"[+] Report saved to: {report_output_path}")


[+] Blended SAR overlay saved to: ../data/output/change_maps/sar_overlay_blended.png
[+] Report saved to: ../reports/sar_intelligence_report.md
